In [6]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

In [7]:
pdb = PDBFile('/home/defense/leonfounlin/aMD_diff_Ab/collec_ab/clean_1IGT.pdb')

In [8]:
forcefield = ForceField('charmm36.xml','charmm36/spce.xml')

In [9]:
modeller = Modeller(pdb.topology,pdb.positions)
modeller.deleteWater()
residues = modeller.addHydrogens(forcefield)

In [ ]:
modeller.addSolvent(forcefield,padding=0.1*nanometer)

KeyboardInterrupt: 

In [ ]:
platform = Platform.getPlatformByName('CUDA')

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=2.0*nanometer, constraints=HBonds)
integrator = CompoundIntegrator()
integrator.addIntegrator(LangevinIntegrator(300,1/picoseconds,2*femtoseconds))
integrator.addIntegrator(AMDIntegrator(0.002*picoseconds, 5, -9280669.89706166))
simulation = Simulation(modeller.topology, system, integrator,platform)
simulation.context.setPositions(modeller.positions)

In [ ]:
print("Minimizing energy")
simulation.minimizeEnergy()

Minimizing energy


In [ ]:
simulation.reporters.append(PDBReporter('/home/defense/leonfounlin/aMD_diff_Ab/collec_ab/aoutput.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))
simulation.reporters.append(StateDataReporter("/home/defense/leonfounlin/aMD_diff_Ab/collec_ab/amd_log.txt", 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

In [ ]:
print("Running NVT")
integrator.setCurrentIntegrator(0)
simulation.step(50000)

Running NVT
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
1000,-904210.3466966948,255.98942024754734,594.6449738947617
2000,-879559.9658373198,294.0935203250811,594.6449738947617
3000,-872153.7920091948,298.38617328558433,594.6449738947617
4000,-870810.8545091948,297.04426164150107,594.6449738947617
5000,-870626.0302904448,301.6485024855213,594.6449738947617
6000,-870559.9384935698,301.95061066402656,594.6449738947617
7000,-871379.4248216948,300.9304337158941,594.6449738947617
8000,-871530.6670091948,300.0647963799298,594.6449738947617
9000,-870570.1689623198,298.7971690844735,594.6449738947617
10000,-870332.2568529448,298.34887174468986,594.6449738947617


KeyboardInterrupt: 

In [ ]:
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
simulation.context.reinitialize(preserveState=True)

In [ ]:
simulation.reporters.append(DCDReporter('/home/defense/leonfounlin/aMD_diff_Ab/collec_ab/atrajectory_npt.dcd',100,enforcePeriodicBox=True))

In [ ]:
print("Running NPT")
integrator.setCurrentIntegrator(1)
simulation.step(100000000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import statistics as stc

In [ ]:
with open('amd_log.txt', 'r') as f :
    data = [i.strip() for i in f.readlines()]
    print(data[0].split(','))
    data = [i.split(',') for i in data[1:]]
    data_array = np.array(data).astype('float')
    potential = data_array[:,1][600:]
    temperature = data_array[:,2][600:]
plt.plot(potential)

In [ ]:
plt.plot(temperature)

In [ ]:
system.getNumParticles()

In [ ]:
def comp_P(V):
    kb = 1,380649*10-23
    N = 59328
    T = 300
    P = (N*kb*T)/V
    return P

In [ ]:
stc.mean(potential)